In [4]:
import os
import random
import pandas as pd #excel可以做的圖表pandas都做得到
import numpy as np #矩陣運算處理用numpy
from PIL import Image


In [5]:
#讀檔

root_dir=os.path.abspath('C:\\Users\\DELL\\Desktop\\ML_final_project')
train=pd.read_csv(os.path.join(root_dir,'train.csv'))
test=pd.read_csv(os.path.join(root_dir,'test.csv'))

#train=pd.read_csv('C:\\Users\\DELL\\Desktop\\ML_final_project\\train.csv')
#test=pd.read_csv('C:\\Users\\DELL\\Desktop\\ML_final_project\\test.csv')
 
print(train.head())
print(test.head())

type(train)

#train.head()

           name  age
0  ML000001.jpg    0
1  ML000002.jpg    0
2  ML000003.jpg    0
3  ML000004.jpg    0
4  ML000005.jpg    0
   id          name
0   1  ML159443.jpg
1   2  ML156219.jpg
2   3  ML159875.jpg
3   4  ML160305.jpg
4   5  ML156123.jpg


pandas.core.frame.DataFrame

In [6]:
#格式化圖片尺寸32*32 + 將圖片轉換成numpy數組

temp=[]
for img_name in train.name:
    img_path=os.path.join(root_dir,'train',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
train_x=np.stack(temp)
print(train_x.shape)
print(train_x.ndim) #ndim-->多少維度


(152552, 32, 32, 3)
4


In [7]:
temp=[]
for img_name in test.name:
    img_path=os.path.join(root_dir,'test',img_name)
    img=Image.open(img_path)
    img=img.resize((32,32))
    array=np.array(img)
    temp.append(array.astype('float32'))
test_x=np.stack(temp)
print(test_x.shape)
print(test_x.ndim)

(10896, 32, 32, 3)
4


In [8]:
#每個像素是0-255的值，影像資料標準化：針對灰階image最簡單的方式是除以255。最後print第0筆資料，發現其值已介於0與1之間。
#normalization-->讓模型訓練更快

train_x = train_x / 255.
test_x = test_x / 255.

In [9]:
#圖片年齡大致分布
#0:10-19歲 、1:20-29歲 、2:30-39歲、 3:40-49歲、4:50-59歲、 5:60歲以上

train.age.value_counts(normalize=True)

2    0.263726
3    0.246827
1    0.239027
4    0.189424
0    0.043179
5    0.017817
Name: age, dtype: float64

In [10]:
#將目標變量(在此專題指age)處理虛擬列(dummy columns)-->使模型更容易接受識別它(be easier for our model to ingest it)
#Many machine learning tools will only accept numbers as input. 
#This may be a problem if you want to use such tool but your data includes categorical features. 

import keras
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()
train_y=lb.fit_transform(train.age)
print(train_y)
train_y=keras.utils.np_utils.to_categorical(train_y)
print(train_y)
print(train_y.shape)

[0 0 0 ... 5 5 5]
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
(152552, 6)


In [34]:

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential, Model
from keras.layers import *
from keras.layers.advanced_activations import PReLU, LeakyReLU, ELU
from keras.optimizers import SGD
from keras.constraints import maxnorm

from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense,Flatten,InputLayer
from keras.layers import Dropout,Flatten,Conv2D,MaxPooling2D

epochs =10

model = Sequential([
        BatchNormalization(input_shape = (32,32,3)),
        Convolution2D(32,(3,3), activation='sigmoid'),
        LeakyReLU(alpha = 0.3),
        BatchNormalization(),
        Convolution2D(32,(3,3), activation='sigmoid'),
        LeakyReLU(alpha = 0.3),
        BatchNormalization(),
        MaxPooling2D(),
        Convolution2D(64,(3,3), activation='sigmoid'),
        LeakyReLU(alpha = 0.3),
        BatchNormalization(),
        Convolution2D(64,(3,3), activation='sigmoid'),
        LeakyReLU(alpha = 0.3),
        BatchNormalization(),
        MaxPooling2D(),
        Flatten(),
        Dropout(0.3),
        Dense(384, activation='linear'),
        LeakyReLU(alpha = 0.3),
        Dropout(0.6),
        Dense(6, activation='softmax')
        ])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_26 (Batc (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 30, 30, 32)        896       
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 30, 30, 32)        0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
leaky_re_lu_27 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 28, 28, 32)        128       
__________

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=epochs, validation_split=0.2)


Train on 122041 samples, validate on 30511 samples
Epoch 1/10
122041/122041 [==============================] - 1952s 16ms/step - loss: 1.3315 - acc: 0.3667 - val_loss: 4.8651 - val_acc: 0.0000e+00
Epoch 2/10
122041/122041 [==============================] - 1985s 16ms/step - loss: 1.2589 - acc: 0.3896 - val_loss: 4.9307 - val_acc: 0.0000e+00
Epoch 3/10
122041/122041 [==============================] - 2095s 17ms/step - loss: 1.2427 - acc: 0.4056 - val_loss: 4.9366 - val_acc: 0.0000e+00
Epoch 4/10
122041/122041 [==============================] - 1974s 16ms/step - loss: 1.2282 - acc: 0.4142 - val_loss: 5.1523 - val_acc: 0.0000e+00
Epoch 5/10
122041/122041 [==============================] - 2223s 18ms/step - loss: 1.2144 - acc: 0.4269 - val_loss: 5.0426 - val_acc: 0.0000e+00
Epoch 6/10
122041/122041 [==============================] - 2242s 18ms/step - loss: 1.2027 - acc: 0.4351 - val_loss: 4.9254 - val_acc: 0.0000e+00
Epoch 7/10
122041/122041 [==============================] - 2075s 17ms/st

In [36]:
pred=model.predict_classes(test_x)
pred=lb.inverse_transform(pred)
print(pred)
test['age']=pred
test.to_csv('sub04.csv',index=False)


[3 1 2 ... 3 2 2]


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [33]:
i = random.choice(train.index)
img_name = train.name[i]
 
img=Image.open(os.path.join(root_dir,'Train',img_name))
img.show()
pred = model.predict_classes(train_x)
print('Original:', train.age[i], 'Predicted:', lb.inverse_transform(pred[i]))

Original: 4 Predicted: 2


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
